In [ ]:
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
observation = pd.read_csv("./094/observation.csv", sep='\t', engine="python")

In [ ]:
X_val = observation.drop(columns=['oximetry'], axis=1)

y = observation['oximetry'].values
X = X_val.values
feature_names = X_val.columns

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

## 3.1 A

In [ ]:
def entropy(y):
    """Энтропия множества меток y."""
    counts = np.bincount(y)
    probs = counts[counts > 0] / len(y)
    return -np.sum(probs * np.log2(probs))


def information_gain(y, y_left, y_right):
    """Прирост информации от разбиения y на y_left и y_right."""
    H_before = entropy(y)
    n = len(y)
    H_after = (len(y_left) / n) * entropy(y_left) + (len(y_right) / n) * entropy(y_right)
    return H_before - H_after


In [ ]:
class TreeNode:
    def __init__(self, *, 
                 feature_index=None, 
                 threshold=None, 
                 left=None, 
                 right=None, 
                 value=None):
        """
        feature_index: индекс признака, по которому делим
        threshold: порог (для числовых признаков) или категория (для категориальных, можно хранить иначе)
        left, right: дочерние узлы
        value: класс в листе (если узел – лист)
        """
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None

In [ ]:
class ID3Classifier:
    def __init__(self, 
                 max_depth=3, 
                 min_samples_split=2):
        """
        max_depth: максимальная глубина дерева (включая корень)
        min_samples_split: минимальное число объектов в узле, чтобы пытаться дальше делить
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def fit(self, X, y):
        """
        X: pandas DataFrame или numpy array, форма (n_samples, n_features)
        y: вектор меток (array-like)
        """
        X = np.array(X)
        y = np.array(y).astype(int)
        self.n_classes_ = len(np.unique(y))
        self.n_features_ = X.shape[1]
        self.root = self._grow_tree(X, y, depth=1)
        return self

    def _grow_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))

        # условия остановки
        if (depth >= self.max_depth or 
            num_labels == 1 or 
            num_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return TreeNode(value=leaf_value)

        # ищем лучшее разбиение
        best_feature, best_threshold, best_gain = self._best_split(X, y)
        
        # если прирост информации 0, делить дальше смысла нет
        if best_gain == 0 or best_feature is None:
            leaf_value = self._most_common_label(y)
            return TreeNode(value=leaf_value)

        # делим данные на левую и правую часть
        indices_left = X[:, best_feature] <= best_threshold
        X_left, y_left = X[indices_left], y[indices_left]
        X_right, y_right = X[~indices_left], y[~indices_left]

        # рекурсивно строим поддеревья
        left_child = self._grow_tree(X_left, y_left, depth + 1)
        right_child = self._grow_tree(X_right, y_right, depth + 1)

        return TreeNode(feature_index=best_feature, 
                        threshold=best_threshold, 
                        left=left_child, 
                        right=right_child)

    def _best_split(self, X, y):
        best_gain = 0.0
        best_feature = None
        best_threshold = None
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            X_column = X[:, feature_index]
            # рассматриваем уникальные значения признака как кандидаты порога
            thresholds = np.unique(X_column)

            for threshold in thresholds:
                left_indices = X_column <= threshold
                right_indices = X_column > threshold

                if left_indices.sum() == 0 or right_indices.sum() == 0:
                    continue

                y_left, y_right = y[left_indices], y[right_indices]
                gain = information_gain(y, y_left, y_right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold, best_gain

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        X = np.array(X)
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature_index] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)


In [ ]:
id3 = ID3Classifier(max_depth=3, min_samples_split=2)
id3.fit(X_train, y_train)

# предсказания
y_pred_test = id3.predict(X_test)
y_pred_train = id3.predict(X_train)

# метрики на test
# acc_test = accuracy_score(y_test, y_pred_test)
# prec_test = precision_score(y_test, y_pred_test, average='binary')  # или 'macro', если классов >2
# rec_test = recall_score(y_test, y_pred_test, average='binary')
# 
# print("ID3 - TEST:")
# print("Accuracy :", acc_test)
# print("Precision:", prec_test)
# print("Recall   :", rec_test)

# метрики на train (для проверки overfitting)
# acc_train = accuracy_score(y_train, y_pred_train)
# prec_train = precision_score(y_train, y_pred_train, average='binary')
# rec_train = recall_score(y_train, y_pred_train, average='binary')
# 
# print("\nID3 - TRAIN:")
# print("Accuracy :", acc_train)
# print("Precision:", prec_train)
# print("Recall   :", rec_train)

## 3.1 B

In [ ]:
acc_test = accuracy_score(y_test, y_pred_test)
prec_test = precision_score(y_test, y_pred_test, average='binary')  # или 'macro', если классов >2
rec_test = recall_score(y_test, y_pred_test, average='binary')

print("ID3 - TEST:")
print("Accuracy :", acc_test)
print("Precision:", prec_test)
print("Recall   :", rec_test)

## 3.1 C

In [ ]:
acc_train = accuracy_score(y_train, y_pred_train)
prec_train = precision_score(y_train, y_pred_train, average='binary')
rec_train = recall_score(y_train, y_pred_train, average='binary')

print("\nID3 - TRAIN:")
print("Accuracy :", acc_train)
print("Precision:", prec_train)
print("Recall   :", rec_train)

## 3.2 A

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(
    criterion='entropy',   # чтобы быть ближе к ID3
    max_depth=3,           # можно менять
    random_state=42
)

dt.fit(X_train, y_train)

y_pred_dt_train = dt.predict(X_train)
y_pred_dt_test = dt.predict(X_test)

In [ ]:
acc_train_1 = accuracy_score(y_train, y_pred_dt_train)
acc_train_2 = accuracy_score(y_test, y_pred_dt_test)

In [ ]:
acc_train_1

In [ ]:
acc_train_2

## 3.2 B

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)

log_reg.fit(X_train, y_train)

y_pred_lr_train = log_reg.predict(X_train)
y_pred_lr_test = log_reg.predict(X_test)

# results.append(
#     evaluate_model("LogisticRegression", 
#                    y_train, y_pred_lr_train, 
#                    y_test, y_pred_lr_test,
#                    average='binary')  # или 'macro'
# )


In [ ]:
acc_train_1 = accuracy_score(y_train, y_pred_dt_train)
acc_train_2 = accuracy_score(y_test, y_pred_dt_test)

In [ ]:
acc_train_1

In [ ]:
acc_train_2

## 3.2 C

## 3.2 D


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import export_text
import numpy as np

# tree_rules = export_text(dt, feature_names=feature_names)
# print(tree_rules)

plt.figure(figsize=(18, 8))
plot_tree(
    dt,
    feature_names=feature_names,
    class_names=[str(c) for c in np.unique(y_train)],
    filled=True,
    rounded=True
)
plt.title("DecisionTreeClassifier – learned rules")
plt.show()

## 3.2 E
